In [1]:
import networkx as nx
import osmnx as ox
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np

path = 'data/'

In [ ]:
G2 = ox.graph_from_file(path+'Rostov3.osm', retain_all=True, simplify=False)
fig, ax = ox.plot_graph(G2, fig_height=18)

In [ ]:
df_matrix = nx.to_pandas_adjacency(G2, dtype=np.uint8)

In [ ]:
df_matrix.to_csv(path+'Adj_matrix_bbike_map.csv', index=False)

In [ ]:
adj_list = list(nx.generate_adjlist(G2))

In [ ]:
df_list = pd.DataFrame(adj_list, columns = ['row'])
df_list = pd.DataFrame(df_list.row.str.split(' ').tolist())
df_list = df_list.rename(columns={0: 'Source'})

In [ ]:
df_list.to_csv(path+'Adj_list_bbike_map.csv', index=False)

In [17]:
root = ET.parse(path+'Rostov3.osm').getroot()
nodes = root.findall('./node')
nodes = {node.attrib['id'] : {
    'name' : str(*[c.attrib['v'] for c in node if c.attrib['k']=='name']) 
} for node in nodes}
df_nodes = pd.DataFrame(nodes).T

In [3]:
target_map = {
    r'.*(?i)клиника.*' : 'больница',
    r'.*(?i)больница.*' : 'больница',
    r'.*(?i)лицей.*' : 'школа',
    r'.*(?i)школа.*' : 'школа',
    r'.*(?i)гимназия.*' : 'школа',
    r'.*(?i)магазин.*' : 'магазин',
    r'.*(?i)универсам.*' : 'магазин',
}

df_nodes = df_nodes.replace({'name' : target_map}, regex=True)
df_nodes.loc[~df_nodes['name'].isin(['больница', 'школа', 'магазин']), 'name'] = ''

In [4]:
df_nodes.to_csv(path+'Nodes_bbike_map.csv')